# INF-285 / ILI-285
## Desafío 1, v1.01
### SCT 2020-1

### Francis Vargas Ferrer
### 201573026-1

# Introducción

En el siguiente desafío estudiaremos el comportamiento de $2$ algoritmos para obtener el punto fijo $r$ de funciones $g(x)$, es decir, $r=g(r)$.
Es importante destacar que el punto fijo de una función no es lo mismo que la raíz de una función, sin embargo sí están muy relacionados.
Solo a modo de recordatorio, la raíz de una función $f(x)$ es encontrar un $\hat{x}$ tal que $f(\hat{x})=0$.

## Iteración de Punto Fijo

El algoritmo llamado Iteración de Punto Fijo (IPF o *FPI*, *Fixed Point Iteration* del inglés) se define de la siguiente forma:
\begin{align*}
  x_0 &= \text{"Initial guess''},\\
  x_{i+1} &= g(x_i), \quad i\in {1,2,3,\dots}.
\end{align*}

El cual puede o no puede converger a su punto fijo $r=g(r)$ dependiendo del comportamiento de $g(x)$ entorno al punto fijo $r$.
En el caso de que la iteración de punto fijo diverja, uno debiera buscar otra forma de encontrar el punto fijo, la otra manera se explica a continuación.

## Método de la Bisección

En el caso de que la iteración de punto fijo diverja o simplemente converja muy lento, podemos usar convenientemente el Método de la Bisección.
Para poder utilizar el Método de la Bisección, debemos adaptarlo, dado que es un algoritmo diseñado para buscar raíces de una función, no puntos fijos de una función.
La adaptación consiste en escribir convenientemente la búsqueda de un punto fijo como la búsqueda de una raíz de la siguiente forma,
\begin{equation}
  f(x) = x - g(x),
\end{equation}

donde podemos comprobar que si evaluamos la función $f(x)$ en el punto fijo de $g(x)$ obtenemos la equivalencia,
\begin{equation}
  f(r) = r - g(r)=0.
\end{equation}

Por lo tanto, ¡hemos exitosamente conectado un problema de punto fijo con un problema de búsqueda de ceros!

**De esta forma ambos métodos podrían ser útiles si necesitamos encontrar puntos fijos de funciones**.

Comentario: ¿Puede visualizar ahora el como utilizar búsqueda de puntos fijos para encontrar raíces de funciones?

# Ejercicio


In [1]:
# Bibliotecas necesarias
import numpy as np
import matplotlib.pyplot as plt

Se solicita implementar una rutina ```obtener_punto_fijo``` que reciba la función $g(x)$, un intervalo $[a, b]$ y un ```n_iter```, que indica el máximo número de iteraciones que pueden utilizar los métodos de bisección y punto fijo.
Notar que los métodos deben retornar la secuencia de soluciones obtenidas hasta que se logra la convergencia, es necesario que cuando se logre el punto fijo no se retorne una secuencia de valores repetidos, si no que se trunque el vector de salida hasta donde empezó a repetirse el valor respectivo, de otra forma se estará dividiendo por $0$ en la explicación incluida más adelante.

El retorno de la rutina debe ser la mejor solución aproximada ```x_sol```, y una estructura del tipo 
```[('biseccion', tasa_bisección), ('punto fijo', tasa_punto_fijo)]```, donde se reporta el algoritmo (en el orden solicitado) y la tasa de convergencia respectiva.
Por lo tanto la firma de la función debería quedar como:
```python
  def obtener_punto_fijo(g, a, b, n_iter):
    # Su algoritmo...

    resultado = [('biseccion', tasa_biseccion), ('punto fijo', tasa_punto_fijo)]
    x_sol = ...
    return x_sol, resultado
```

La idea es que su algoritmo permita retornar la solución asociada al método con mejor *tasa de convergencia*.

Para que pueda calcular la *tasa de convergencia* se pone a disposición la función ```obtener_tasa(ratio)```, que recibe un arreglo con los cocientes de la estimación numérica de los errores en cada iteración. Los cuales deben ser obtenidos de la siguiente forma:
\begin{equation}
  ratio_i = \frac{|x_{i+1} - x_i|}{|x_i - x_{i-1}|}
\end{equation}

In [2]:
def obtener_tasa(ratio):
  hist, bin_edges = np.histogram(ratio, bins=10000)
  k = np.argmax(hist)
  return np.round((bin_edges[k] + bin_edges[k+1]) / 2, 5)

Además, para que pueda probar el funcionamiento de su procedimiento, se ponen a disposición las siguientes funciones y los intevalos donde debe buscar el punto fijo:

In [3]:
g1 = lambda x: np.cos(x) # Intervalo: [0, 1]
g2 = lambda x: 3 / (x-2) # Intervalo: [-3, 0]
g3 = lambda x: (x + 10.) ** (1 / 4) # Intervalo: [0, 2]
g4 = lambda x: 3 + 2 * np.sin(x) # Intervalo: [-5, 5]
g5 = lambda x: np.cos(x) / np.exp(x) # Intervalo: [0, 4]
g6 = lambda x: (np.exp(x) + x ** 3 + 4 * x ** 2 + 2 * x + 2) / (x ** 2 + 3 * x - 3) # Intervalo: [-1, 0]
g7 = lambda x: np.exp((np.exp(-x) / 3)) # Intervalo: [0, 2]
g8 = lambda x: -0.5 * x + 3 / 2 # Intervalo: [0, 1]
g9 = lambda x: (x ** 3 - 5) / 2 # Intervalo: [2, 3]
g10 = lambda x: -1 + 1.5 * x # Intervalo: [0,10]
g11 = lambda x: 0.7 + 1.7 * x # Intervalo: [-10,10]
fs = [g1, g2, g3, g4 ,g5 ,g6 ,g7, g8, g9, g10, g11]
points = [[0,1], [-3,0], [0,2], [-5,5], [0,4], [-1,0], [0,2], [0,1], [2,3], [0,10], [-10,10]]

Se incluye a continuación el enunciado de la función que usted debe entregar:

In [5]:
def bisection(f, a, b, n_iter):
    x = list()
    g = lambda x: x - f(x) #Definimos nuestra nueva funcion para que encuentre punto fijo y no "raiz".
    if g(a) == 0:#Consideramos casos en los que a o b sean "raiz".
        return [a]
    elif g(b) == 0:
        return [b]
    elif g(a)*g(b) < 0:
        for i in range(0, n_iter): #Itertamos n_iter veces
            c = (a+b)/2
            if len(x)>0 and x[i-1] == c:#Si nuestro ultimo c es igual al ultimo almacenado retornamos para no tener valores repetidos.
                return x
            x.append(c)
            if g(a)*g(c) < 0:
                b = c
            elif g(c)*g(b) < 0:
                a = c
            elif g(c) == 0:
                return x
    return x

In [5]:
def fpi(g, x_0, n_iter):
    # CONSIDERE que el metodo puede no converger
    # RECUERDE no incluir valores repetidos al final de la secuencia del arreglo de salida para no tener errores igual a 0
    x = [0]*(n_iter+1) # Calcular los valores de x para cada iteracion
    x[0] = x_0 #El primer elemento de la lista es nuestro initial guess
    for i in range(1, n_iter+1):
        x[i] = g(x[i-1])
        if i>1:#La primera iteracion no tiene error
            error = abs(x[i]-x[i-1])#Calculamos el error absoluto para saber si converge o no
            if error > 1e2:#Diverge
                return x[0:i+1]
        if x[i] == x[i-1]:
            return x[0:i+1]#Se retorna el arreglo hasta el ultimo valor cambiado(que no es 0)
    return x


In [6]:
def obtener_punto_fijo(g, a, b, n_iter):
    bisec = bisection(g,a,b,n_iter)
    fp = fpi(g,a,n_iter)

    ratio_bise = [0]
    ratio_fpi = [0]
    for i in range(2, len(bisec)-1):#Calculamos ratios para biseccion
        ratio_bise.append(abs(bisec[i+1]-bisec[i])/abs(bisec[i]-bisec[i-1]))
    for i in range(2, len(fp)-1):#Calculamos ratios para iteracion de punto fijo
        ratio_fpi.append(abs(fp[i+1]-fp[i])/abs(fp[i]-fp[i-1]))
        
    tasa_biseccion = obtener_tasa(ratio_bise)
    tasa_punto_fijo = obtener_tasa(ratio_fpi)
    resultado = [('biseccion', tasa_biseccion), ('punto fijo', tasa_punto_fijo)]
    x_sol = bisec[len(bisec)-1] if ratio_bise[len(ratio_bise)-1] < ratio_fpi[len(ratio_fpi)-1] else fp[len(fp)-1] #La mejor solucion corresponde al metodo con menor tasa
    return x_sol, resultado
